In [ ]:
# def func_gradient(func, theta_sym, values):
#    grad = np.array([diff(L(*theta_sym), i) for i in theta_sym])
#    substitutes = list(zip(theta_sym, values))
#
#    for i, element in enumerate(grad):
#        for j in substitutes:
#            grad[i] = grad[i].subs(*j)
#    return grad  

In [9]:
# from Algorithm 1: HMC, page1353
def Leapfrog(theta, r, eps):
    r_tilde = r + (eps/2) * L_gradient(*theta)
    theta_tilde = theta + eps * r_tilde
    r_tilde = r_tilde + (eps/2) * L_gradient(*theta_tilde)
    return theta_tilde, r_tilde

In [10]:
# from Algorithm 3: Efficeint NUTS pg.1364
def BuildTree(theta, r, u, v, j, eps):
    if j == 0:
        # base case, take one leapfrog step in the direction v
        theta_prime, r_prime = Leapfrog(theta, r, v*eps)
        
        n_prime = int(u <= np.exp(L(*theta_prime) - 0.5 * np.dot(r_prime, r_prime)))
        
        s_prime = int(L(*theta_prime) - 0.5 * np.dot(r_prime, r_prime) > np.log(u) - 1000)
        
        return theta_prime, r_prime, theta_prime, r_prime, theta_prime, n_prime, s_prime
    else:
        # recursion, build left and right subtrees
        theta_minus, r_minus, theta_plus, r_plus, theta_prime, n_prime, s_prime = BuildTree(theta, r, u, v, j-1, eps)
        
        if s_prime == 1:
            if v == -1:
                theta_minus, r_minus, _,_, theta_doub_prime, n_doub_prime, s_doub_prime = BuildTree(theta_minus, r_minus, u, v, j-1, eps)
            else:
                _, _, theta_plus, r_plus, theta_doub_prime, n_doub_prime, s_doub_prime = BuildTree(theta_plus, r_plus, u, v, j-1, eps)

            # Use Metropolis-Hastings
            prob = n_doub_prime / (n_prime + n_doub_prime)
            if (np.random.uniform(0,1,1) < prob):
                theta_prime = theta_doub_prime
            
            ind_1 = int(np.dot(theta_plus-theta_minus, r_minus) >= 0)
            ind_2 = int(np.dot(theta_plus-theta_minus, r_plus) >= 0)
            s_prime = s_doub_prime * ind_1 * ind_2
            n_prime = n_prime + n_doub_prime
        
        return theta_minus, r_minus, theta_plus, r_plus, theta_prime, n_prime, s_prime

In [11]:
def efficient_nuts(theta0, eps, L, M, L_gradient):
    # initialize samples matrix
    # put initial theta0 in first row of matrix
    samples = np.empty((M+1, len(theta0)))
    samples[0,:] = theta0
    
    for m in range(1, M+1):
        # resample
        norm_samp = np.random.multivariate_normal(np.repeat(0, len(theta0)), np.identity(len(theta0)), 1)
        r0 = norm_samp.ravel()
        upper = np.exp(L(*samples[m-1,:]) - 0.5*np.dot(r0, r0))
        u = np.random.uniform(0, upper, 1)
        
        # initialize
        theta_minus = samples[m-1,:]
        theta_plus = samples[m-1,:]
        r_minus = r0
        r_plus = r0
        j = 0
        samples[m,:] = samples[m-1,:]
        n = 1
        s = 1
        
        while s == 1:
            v_j = np.random.uniform(-1,1,1)
            if v_j == -1:
                theta_minus, r_minus, _, _, theta_prime, n_prime, s_prime = BuildTree(theta_minus, r_minus, u, v_j, j, eps)
            else:
                _, _, theta_plus, r_plus, theta_prime, n_prime, s_prime = BuildTree(theta_plus, r_plus, u, v_j, j, eps)
            
            if s_prime == 1:
                # Use Metropolis-Hastings
                prob = min(1, n_prime/n)
                if (np.random.uniform(0,1,1) < prob):
                    samples[m,:] = theta_prime
                    
            n = n + n_prime

            boolean_1 = int(np.dot(theta_plus-theta_minus, r_minus) >= 0)
            boolean_2 = int(np.dot(theta_plus-theta_minus, r_plus) >= 0)
            s = s_prime * boolean_1 * boolean_2
            j = j + 1
    return samples

In [14]:
from scipy.stats import norm
X = np.random.normal(2, 5, size = 100)
n = 100

def L(mu, var):
    log_likelihood = -n/2 * np.log(var) - n/2 * np.log(2*np.pi) - sum((X - mu)**2) / (2*var)
    return log_likelihood

def L_gradient(mu, var):
    one = sum(X-mu) / var
    two = -n/ (2*var) + sum((X-mu)**2)/ (2*var**2)
    return np.array([one, two])


In [19]:
# simple example
theta0 = np.array([5,3])
eps = 0.1
M = 100
results = efficient_nuts(theta0, eps, L, M, L_gradient)
np.mean(results[1])

ZeroDivisionError: division by zero

In [ ]:
# more complicated example

In [ ]:
# try to optimize code

In [ ]:
# comparison with STAN NUTS

In [ ]:
# comparison of NUTS, metropolis, and Gibbs (speed and plots)